In [69]:
import asyncio
import re
import os
import json
import pandas as pd
from datetime import datetime, date
from pathlib import Path
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError

# ============================================================
# CONFIG
# ============================================================
try:
    BASE_DIR = Path(__file__).resolve().parent
except NameError:
    BASE_DIR = Path(os.getcwd()).resolve()

SAVE_DIR = BASE_DIR / "deposit_rate"
SAVE_DIR.mkdir(parents=True, exist_ok=True)
MASTER_PATH = SAVE_DIR / "deposit_rate_all.json"
LOG_PATH = SAVE_DIR / "deposit_log.csv"

# ============================================================
# UTILITIES
# ============================================================

def load_log():
    if LOG_PATH.exists():
        return pd.read_csv(LOG_PATH)
    else:
        cols = ["date", "link", "json_file", "status", "rows", "timestamp"]
        return pd.DataFrame(columns=cols)

def save_log(df):
    df.to_csv(LOG_PATH, index=False, encoding="utf-8-sig")

def save_json(data, path):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

async def safe_goto(page, url, max_retries=3, timeout=60000):
    for attempt in range(1, max_retries + 1):
        try:
            await page.goto(url, timeout=timeout)
            await page.wait_for_load_state("domcontentloaded", timeout=15000)
            return True
        except Exception as e:
            print(f"⚠️ Navigation attempt {attempt}/{max_retries} failed: {e}")
            await asyncio.sleep(2)
    return False

# ============================================================
# TABLE PARSER — dynamic version
# ============================================================

def parse_interest_table(html, fallback_date):
    """Extract deposit rate table with dynamic month handling and 'no_term'."""
    soup = BeautifulSoup(html, "html.parser")

    # --- Detect table date ---
    match = re.search(r"(\d{1,2}/\d{1,2}/\d{4})", soup.get_text(" ", strip=True))
    table_date = (
        datetime.strptime(match.group(1), "%d/%m/%Y").strftime("%Y-%m-%d")
        if match
        else fallback_date
    )

    # --- Find target table ---
    target_table = None
    for tbl in soup.find_all("table"):
        text = tbl.get_text(" ", strip=True).lower()
        if all(k in text for k in ["ngân", "tháng", "lãi suất"]):
            target_table = tbl
            break
    if not target_table:
        print("⚠️ No valid table found in page.")
        return None

    # --- Parse rows ---
    rows = target_table.find_all("tr")
    if len(rows) < 3:
        return None

    header_cells = rows[1].find_all("td")
    headers = ["bank"] + [
        re.sub(r"\s+", "_", c.get_text(strip=True).lower()) for c in header_cells[1:]
    ]

    data = []
    for row in rows[2:]:
        cells = row.find_all("td")
        if len(cells) < 2:
            continue
        bank = cells[0].get_text(strip=True).upper()

        # Extract numeric values per cell
        rates = []
        for c in cells[1:]:
            txt = c.get_text(strip=True).replace(",", ".")
            try:
                val = float(txt)
                if val > 100:
                    val = None
            except ValueError:
                val = None
            rates.append(val)

        record = {"date": table_date, "bank": bank}
        for i, h in enumerate(headers[1:]):
            record[h] = rates[i] if i < len(rates) else None
        data.append(record)

    df = pd.DataFrame(data)

    # --- Normalise columns ---
    rename_map = {}
    for col in df.columns:
        col_clean = col.lower().strip()
        if "tháng" in col_clean:
            # e.g. "24_tháng" → "24m"
            m = re.search(r"(\d+)", col_clean)
            if m:
                rename_map[col] = f"{m.group(1)}m"
        elif "không" in col_clean or "kỳ_hạn" in col_clean:
            rename_map[col] = "no_term"

    df = df.rename(columns=rename_map)

    # --- Clean numeric values ---
    for c in [x for x in df.columns if re.match(r"^\d+m$", x) or x == "no_term"]:
        df[c] = (
            df[c]
            .astype(str)
            .str.replace("%", "", regex=False)
            .str.replace(",", ".", regex=False)
            .str.strip()
        )
        df[c] = pd.to_numeric(df[c], errors="coerce")
        df.loc[df[c] > 100, c] = None

    # --- Final clean ---
    df = df.where(pd.notnull(df), None)
    return df

# ============================================================
# MAIN SCRAPER
# ============================================================

async def scrape_vietnamnet_interest(start_date=None, end_date=None, headless=True):
    df_log = load_log()
    existing_dates = set(df_log["date"].dropna().unique())

    start_dt = datetime.strptime(start_date, "%Y-%m-%d") if start_date else None
    end_dt = datetime.strptime(end_date, "%Y-%m-%d") if end_date else None

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=headless)
        page = await browser.new_page()

        base = "https://vietnamnet.vn"
        page_num = 1
        stop = False

        while True:
            url = f"https://vietnamnet.vn/tim-kiem-p{page_num}?q=Lãi suất ngân hàng hôm nay&od=2&bydaterang=all&newstype=all"
            print(f"🔍 Page {page_num}")
            if not await safe_goto(page, url):
                break
            try:
                await page.wait_for_selector("h3.vnn-title a", timeout=15000)
            except PlaywrightTimeoutError:
                print("⚠️ No search results found.")
                break

            soup = BeautifulSoup(await page.content(), "html.parser")
            links = []

            for a in soup.select("h3.vnn-title a[href]"):
                title = a.get_text(strip=True)
                href = a["href"]
                link = href if href.startswith("http") else base + href
                m = re.search(r"(\d{1,2}/\d{1,2}/\d{4})", title)
                if not m:
                    continue
                date_str = datetime.strptime(m.group(1), "%d/%m/%Y").strftime("%Y-%m-%d")
                if date_str in existing_dates:
                    continue
                dt = datetime.strptime(date_str, "%Y-%m-%d")
                if start_dt and dt < start_dt:
                    stop = True
                    break
                if end_dt and dt > end_dt:
                    continue
                links.append((date_str, link))

            if stop or not links:
                break

            for date_str, link in links:
                print(f"📄 Parsing {date_str} → {link}")
                key = datetime.strptime(date_str, "%Y-%m-%d").strftime("%y%m%d")
                json_path = SAVE_DIR / f"deposit_rate_{key}.json"
                status, rows = "fail", 0
                try:
                    if await safe_goto(page, link):
                        await page.wait_for_selector("table", timeout=15000)
                        html = await page.content()
                        df = parse_interest_table(html, date_str)
                        if df is not None and not df.empty:
                            data = df.to_dict(orient="records")
                            save_json(data, json_path)
                            rows = len(data)
                            status = "success"

                            # Update master JSON
                            if MASTER_PATH.exists():
                                master = json.load(open(MASTER_PATH, "r", encoding="utf-8"))
                            else:
                                master = []
                            master.extend(data)
                            save_json(master, MASTER_PATH)
                        else:
                            status = "no_table"
                except Exception as e:
                    print(f"⚠️ Error parsing {link}: {e}")
                    status = "fail"

                df_log.loc[len(df_log)] = [
                    date_str,
                    link,
                    json_path.name,
                    status,
                    rows,
                    datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                ]
                save_log(df_log)

            page_num += 1
            await asyncio.sleep(1.2)

        await page.close()
        await browser.close()

    print("✅ Done. Logs synced and data saved.")

# ============================================================
# RUNNER
# ============================================================

def run_scraper(start_date=None, end_date=None, headless=True):
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            return asyncio.ensure_future(scrape_vietnamnet_interest(start_date, end_date, headless))
        else:
            loop.run_until_complete(scrape_vietnamnet_interest(start_date, end_date, headless))
    except RuntimeError:
        asyncio.run(scrape_vietnamnet_interest(start_date, end_date, headless))


# ============================================================
# MANUAL TEST EXAMPLE
# ============================================================

# if __name__ == "__main__":
#     run_scraper(start_date="2025-09-01", end_date="2025-10-25", headless=True)

In [70]:
await run_scraper()

🔍 Page 1
📄 Parsing 2025-09-26 → https://vietnamnet.vn/lai-suat-ngan-hang-hom-nay-26-9-2025-ky-han-duoi-6-thang-cao-nhat-4-7-nam-2446223.html
📄 Parsing 2025-09-25 → https://vietnamnet.vn/lai-suat-ngan-hang-hom-nay-25-9-2025-lai-suat-huy-dong-9-nam-danh-cho-ai-2445835.html
📄 Parsing 2025-09-24 → https://vietnamnet.vn/lai-suat-ngan-hang-hom-nay-24-9-2025-ba-ngan-hang-co-lai-suat-tu-6-nam-2445539.html
📄 Parsing 2025-09-23 → https://vietnamnet.vn/lai-suat-ngan-hang-hom-nay-23-9-2025-nha-bang-tang-lai-suat-len-6-1-nam-2445235.html
📄 Parsing 2025-09-22 → https://vietnamnet.vn/lai-suat-ngan-hang-hom-nay-22-9-2025-gui-5-ty-dong-nhan-lai-6-5-the-nao-2444831.html
📄 Parsing 2025-09-19 → https://vietnamnet.vn/lai-suat-ngan-hang-hom-nay-19-9-2025-nha-bang-huy-dong-von-tra-lai-suat-7-nam-2444069.html
📄 Parsing 2025-09-18 → https://vietnamnet.vn/lai-suat-ngan-hang-hom-nay-18-9-2025-so-sanh-lai-tien-gui-o-nhom-ngan-hang-big4-2443723.html
📄 Parsing 2025-09-17 → https://vietnamnet.vn/lai-suat-ngan-hang-h

In [ ]:
import os
import requests
import pandas as pd
from datetime import datetime
from pathlib import Path

# ======================================================
# CONFIG
# ======================================================
HEADERS = {"User-Agent": "Mozilla/5.0"}
SAVE_DIR = Path.cwd() / "gold_price"
SAVE_DIR.mkdir(parents=True, exist_ok=True)

CSV_FULL = SAVE_DIR / "gold_price_full_history.csv"
CSV_DAILY = SAVE_DIR / "gold_price_daily.csv"

# ======================================================
# 1️⃣ GOLD BAR (SJC miếng) — ALL TIME
# ======================================================
print("🟡 Fetching SJC gold bar data...")
url_bar = "https://cafef.vn/du-lieu/Ajax/ajaxgoldpricehistory.ashx?index=all"
resp_bar = requests.get(url_bar, headers=HEADERS, timeout=30)
resp_bar.raise_for_status()
data_bar = resp_bar.json().get("Data", {})

bar_hist = data_bar.get("goldPriceWorldHistories", [])
bar_records = [
    {
        "date": pd.to_datetime(item.get("createdAt") or item.get("lastUpdated"), errors="coerce"),
        "bar_buy": item.get("buyPrice"),
        "bar_sell": item.get("sellPrice"),
    }
    for item in bar_hist
]
df_bar = pd.DataFrame(bar_records).dropna(subset=["date"]).sort_values("date")

# ======================================================
# 2️⃣ GOLD RING (SJC nhẫn) — ALL TIME
# ======================================================
print("🔶 Fetching SJC gold ring data...")
url_ring = "https://cafef.vn/du-lieu/Ajax/AjaxGoldPriceRing.ashx?time=all&zone=11"
resp_ring = requests.get(url_ring, headers=HEADERS, timeout=30)
resp_ring.raise_for_status()
data_ring = resp_ring.json().get("Data", {})

ring_hist = data_ring.get("goldPriceWorldHistories", [])
ring_records = [
    {
        "date": pd.to_datetime(item.get("createdAt") or item.get("lastUpdated"), errors="coerce"),
        "ring_buy": item.get("buyPrice"),
        "ring_sell": item.get("sellPrice"),
    }
    for item in ring_hist
]
df_ring = pd.DataFrame(ring_records).dropna(subset=["date"]).sort_values("date")

# ======================================================
# 3️⃣ NORMALISE TIMEZONE AND MERGE
# ======================================================
df_bar["date"] = pd.to_datetime(df_bar["date"]).dt.tz_localize(None)
df_ring["date"] = pd.to_datetime(df_ring["date"]).dt.tz_localize(None)

df = pd.merge(df_bar, df_ring, on="date", how="outer").sort_values("date")

# ======================================================
# 4️⃣ CALCULATE SPREADS AND GAPS
# ======================================================
df["bar_spread"] = df["bar_sell"] - df["bar_buy"]
df["ring_spread"] = df["ring_sell"] - df["ring_buy"]
df["bar_ring_gap"] = df["bar_sell"] - df["ring_sell"]

# ======================================================
# 5️⃣ CLEAN DATA
# ======================================================
df = df.dropna(subset=["bar_buy", "bar_sell", "ring_buy", "ring_sell"], how="all")
df["date"] = pd.to_datetime(df["date"]).dt.strftime("%Y-%m-%d %H:%M")
df = df.sort_values("date").reset_index(drop=True)

print(f"✅ Cleaned data — {len(df)} rows\n")

# ======================================================
# 6️⃣ SAVE FULL DATA TO CSV
# ======================================================
df.to_csv(CSV_FULL, index=False, encoding="utf-8-sig")
print(f"💾 Saved full history → {CSV_FULL}")

# ======================================================
# 7️⃣ AGGREGATE DAILY (LAST NON-NULL VALUES)
# ======================================================
df["date_only"] = pd.to_datetime(df["date"]).dt.date
df_daily = (
    df.groupby("date_only")
    .agg({
        "bar_buy": "last",
        "bar_sell": "last",
        "ring_buy": "last",
        "ring_sell": "last"
    })
    .reset_index()
)

# Recalculate spreads and gaps
df_daily["bar_spread"] = df_daily["bar_sell"] - df_daily["bar_buy"]
df_daily["ring_spread"] = df_daily["ring_sell"] - df_daily["ring_buy"]
df_daily["bar_ring_gap"] = df_daily["bar_sell"] - df_daily["ring_sell"]

df_daily = df_daily.rename(columns={"date_only": "date"})
df_daily["date"] = pd.to_datetime(df_daily["date"]).dt.strftime("%Y-%m-%d")
df_daily = df_daily.sort_values("date").reset_index(drop=True)

# ======================================================
# 8️⃣ SAVE DAILY DATA TO CSV
# ======================================================
df_daily.to_csv(CSV_DAILY, index=False, encoding="utf-8-sig")
print(f"💾 Saved aggregated daily data → {CSV_DAILY}")
print(f"✅ Aggregated {len(df_daily)} days total\n")

🟡 Fetching SJC gold bar data...
🔶 Fetching SJC gold ring data...
✅ Cleaned data — 1283 rows

💾 Saved full history → /Users/leeboo/Library/CloudStorage/OneDrive-Personal/Personal data/Study/Trading projects/Webscrapping data/market_data/bank_interest_rate/gold_price/gold_price_full_history.csv
💾 Saved aggregated daily data → /Users/leeboo/Library/CloudStorage/OneDrive-Personal/Personal data/Study/Trading projects/Webscrapping data/market_data/bank_interest_rate/gold_price/gold_price_daily.csv
✅ Aggregated 791 days total

💰 Latest update:
📅 Date: 2025-10-24
🏅 SJC gold bar sell:  148.5 triệu đồng/lượng
💍 SJC gold ring sell: 14850.0 triệu đồng/lượng
🔸 Gap (bar - ring):   -14701.50 triệu đồng/lượng
